## Reverse Geocoding, Preparation for Brushing and Scatter Plot
We have applied reverse geocoding in order to get the location such as county and city from the pairs of geographic coordinates in our dataset. We have then prepared the datasets for brushing. The dataset consist of following two dataframes for the conditions (weather, speed, light and road condition) in ten most populous counties in UK:

##### Data per county: 
This dataset consists of the number of accidents per county under a certain condition. There are columns for each condition per county which includes the number of accidents and also there is a column for total that represents the total number of accidents in that county
##### Data per county per year:
This datase consists of the number of accidents per county under a certain condition for the years 2009 to 2014. This dataset has columns for each condition per county per year, and as well as the column for total number of accidents.

In order to make the notebook readible to the audience, the data preparation for speed limit is only included in the notebook. It should be noted that the data for the other conditions is also prepared in the exact same way. 

# Reverse GeoCoding

In [1]:
import reverse_geocoder as rg 
import pprint 
import pandas as pd

In [ ]:
gdf= pd.read_csv("CleanData_UK_Accidents.csv")

In [ ]:
gdf.head()

In [ ]:
latlong= gdf[['Latitude','Longitude']]

In [ ]:
tuples = [tuple(x) for x in latlong.values]

In [ ]:
result = rg.search(tuples) 
res=pd.DataFrame(result) 

In [ ]:
res.head()

In [ ]:
resultpd=res[['name','admin1','admin2']]

In [ ]:
resultpd.head()

In [ ]:
data=pd.concat([gdf, resultpd], axis=1, sort=False)

In [ ]:
data.rename(columns={'admin1':'Country',
                          'admin2':'Area'}, 
                 inplace=True)
del data['Unnamed: 0']

In [ ]:
data.head()

data.to_csv('Clean_data_with_City_Names.csv', index=False)


# Prep for Brushing and Scatter plot

In [ ]:
most_pop=['Greater London','Manchester','Hampshire','Kent','Essex','Lancashire','Devon','Surrey','Hertfordshire','North Yorkshire']

In [ ]:
data = data[data["Area"].isin(most_pop)]

In [ ]:
xy=['Road_Surface_Conditions','Weather_Conditions','Light_Conditions','Speed_limit']

In [ ]:
xyvalues=xy[3]
speed=data[xyvalues].unique().tolist()

In [ ]:
county_df = data[['Area']].drop_duplicates().dropna()
county_df

In [ ]:
Area_df = data[['Area']].drop_duplicates().dropna()
for w in (speed):
   
    df1= (data.loc[data[xyvalues] == w])
    
    
    df1=df1.groupby(['Area']).aggregate('count').reset_index()
    
    df1=df1[['Area',xyvalues]]
   
    df1 = df1.rename(columns = {xyvalues:w})
    Area_df = pd.merge(Area_df,df1,on = "Area",how="outer")
    Area_df.fillna(0, inplace=True)
sumPD_year = pd.DataFrame(Area_df.iloc[:,2:15].sum(axis=1))

Area_df=pd.concat([Area_df, sumPD_year], axis=1, sort=False)

Area_df.rename(columns={0:'Total'}, 
                 inplace=True)
Area_df    

In [ ]:
Area_year_DF= data[["Area","Year"]].drop_duplicates().dropna()
for w in (speed):    
    df2= (data.loc[data[xyvalues] == w])
    df2=df2.groupby(['Area','Year']).aggregate('count').reset_index()
    df2=df2[['Area','Year',xyvalues]]
    df2 = df2.rename(columns = {xyvalues:w})
    Area_year_DF = pd.merge(Area_year_DF,df2, on =["Area","Year"],how="left")
    #Area_year_DF.fillna(0, inplace=True)
sumPD_year = pd.DataFrame(Area_year_DF.iloc[:,2:15].sum(axis=1))

Area_year_DF=pd.concat([Area_year_DF, sumPD_year], axis=1, sort=False)

Area_year_DF.rename(columns={0:'Total'}, 
                 inplace=True)
Area_year_DF